# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [5]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [6]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

In [7]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [8]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population']).fillna(0)
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population']).fillna(0)

In [9]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [10]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [11]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [12]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [13]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [14]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [15]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [16]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [17]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [18]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [19]:
drop_cats = ['source_type', 'source_class',
              'water_quality', 'quality_group',
              'quantity_group', 'waterpoint_type_group',
              'payment_type', 'extraction_type_group',
              'extraction_type_class', 'management_group'
              ]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [20]:
X_train.iloc[0,:]

amount_tsh                                              6000
funder                                                 Roman
gps_height                                              1390
installer                                              Roman
longitude                                            34.9381
latitude                                            -9.85632
wpt_name                                                   1
basin                                             Lake Nyasa
subvillage                                          Mnyusi B
region                                                Iringa
region_code                                               11
district_code                                              5
lga                                                   Ludewa
ward                                                Mundindi
population                                               109
public_meeting                                             1
scheme_management       

In [21]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []

In [22]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [23]:
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = TfidfVectorizer(ngram_range=(1,2))
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    # kbest = SelectKBest(chi2, min(15, tr.shape[1])).fit(tr, ytr)
    sel = SelectFromModel(Logistic)
    tr = kbest.transform(tr)
    te = kbest.transform(te)
    tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    text_feats_train.append(tr)
    text_feats_test.append(te)



X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore')] + text_feats_train,
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] + text_feats_test,
                    axis=1)


In [24]:
X_train.head()

,amount_tsh,gps_height,longitude,latitude,wpt_name,population,public_meeting,permit,construction_year,date_recorded_year,...,waterpoint_type_5,waterpoint_type_6,waterpoint_type_7,waterpoint_type_8,waterpoint_type_9,waterpoint_type_10,waterpoint_type_11,waterpoint_type_12,waterpoint_type_13,waterpoint_type_14
0,6000.0,1390,34.93809275,-9.85632177,1,109,1.0,0.0,1999,2011,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.549138,0.308775,0.000000,0.0
1,0.0,1399,34.6987661,-2.14746569,1,280,0.5,1.0,2010,2013,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.549138,0.308775,0.000000,0.0
2,25.0,686,37.46066446,-3.82132853,1,250,1.0,1.0,2009,2013,...,0.0,0.420692,0.420692,0.0,0.0,0.0,0.395413,0.000000,0.420692,0.0
3,0.0,263,38.48616088,-11.15529772,1,58,1.0,1.0,1986,2013,...,0.0,0.420692,0.420692,0.0,0.0,0.0,0.395413,0.000000,0.420692,0.0
4,0.0,0,31.13084671,-1.82535885,1,0,1.0,1.0,0,2011,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.549138,0.308775,0.000000,0.0


In [25]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('longitude', dtype('O')),
 ('latitude', dtype('O')),
 ('wpt_name', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('construction_year', dtype('O')),
 ('date_recorded_year', dtype('O')),
 ('date_recorded_month', dtype('O')),
 ('date_recorded_day', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [26]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
#y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
#display(
#    np.where(ytr != 0)[0][:40],
#    np.where(ycv != 0)[0][:40],
#    np.where(yva != 0)[0][:40],)
#ytr = y_bin.transform(ytr)
#ycv = y_bin.transform(ycv)
#yva = y_bin.transform(yva)

In [27]:
def y_to_df(y, ids):
    #y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [28]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'wpt_name',
 'population',
 'public_meeting',
 'permit',
 'construction_year',
 'date_recorded_year',
 'date_recorded_month',
 'date_recorded_day',
 'age',
 'amount_per_man',
 'funder_0',
 'funder_1',
 'funder_2',
 'funder_3',
 'funder_4',
 'funder_5',
 'funder_6',
 'funder_7',
 'funder_8',
 'funder_9',
 'funder_10',
 'funder_11',
 'funder_12',
 'funder_13',
 'funder_14',
 'installer_0',
 'installer_1',
 'installer_2',
 'installer_3',
 'installer_4',
 'installer_5',
 'installer_6',
 'installer_7',
 'installer_8',
 'installer_9',
 'installer_10',
 'installer_11',
 'installer_12',
 'installer_13',
 'installer_14',
 'basin_0',
 'basin_1',
 'basin_2',
 'basin_3',
 'basin_4',
 'basin_5',
 'basin_6',
 'basin_7',
 'basin_8',
 'basin_9',
 'basin_10',
 'basin_11',
 'basin_12',
 'basin_13',
 'basin_14',
 'subvillage_0',
 'subvillage_1',
 'subvillage_2',
 'subvillage_3',
 'subvillage_4',
 'subvillage_5',
 'subvillage_6',
 'subvillage_7',
 

In [29]:
X_train.to_csv('X_train_prepr2.csv')
X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
pd.Series(ytr).to_csv('y_train_prepr2.csv')
pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [30]:
import catboost

In [ ]:
clf = catboost.CatBoostClassifier(1800, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importances=True)

In [ ]:
clf.fit(xtr, ytr, cat_features=[xtr.columns.get_loc(c) for c in text_features if c in xtr.columns],
        eval_set=(xcv, ycv), verbose=True)

Borders generated
0:	learn 0.7057338015	test 0.7046475079	bestTest 0.7046475079	passed: 0.308 sec	total: 898ms	remaining: 26m 54s
1:	learn 0.7058059551	test 0.7063313875	bestTest 0.7063313875	passed: 0.293 sec	total: 1.19s	remaining: 17m 50s
2:	learn 0.7084275338	test 0.7089133363	bestTest 0.7089133363	passed: 0.374 sec	total: 1.56s	remaining: 15m 37s
3:	learn 0.705397085	test 0.7047597665	bestTest 0.7089133363	passed: 0.282 sec	total: 1.85s	remaining: 13m 49s
4:	learn 0.7050603685	test 0.7046475079	bestTest 0.7089133363	passed: 0.28 sec	total: 2.13s	remaining: 12m 43s
5:	learn 0.7049401126	test 0.7048720251	bestTest 0.7089133363	passed: 0.287 sec	total: 2.41s	remaining: 12m 1s
6:	learn 0.7065274905	test 0.7067804221	bestTest 0.7089133363	passed: 0.301 sec	total: 2.71s	remaining: 11m 35s
7:	learn 0.7066477464	test 0.7067804221	bestTest 0.7089133363	passed: 0.27 sec	total: 2.98s	remaining: 11m 8s
8:	learn 0.707393333	test 0.7073417153	bestTest 0.7089133363	passed: 0.28 sec	total: 3.27s	

74:	learn 0.7237721872	test 0.7217108217	bestTest 0.721935339	passed: 0.318 sec	total: 22.5s	remaining: 8m 38s
75:	learn 0.7236278801	test 0.7215985631	bestTest 0.721935339	passed: 0.368 sec	total: 22.9s	remaining: 8m 39s
76:	learn 0.7237721872	test 0.7211495285	bestTest 0.721935339	passed: 0.256 sec	total: 23.2s	remaining: 8m 38s
77:	learn 0.7236038289	test 0.7212617872	bestTest 0.721935339	passed: 0.28 sec	total: 23.4s	remaining: 8m 37s
78:	learn 0.7243253644	test 0.7218230804	bestTest 0.721935339	passed: 0.36 sec	total: 23.8s	remaining: 8m 38s
79:	learn 0.7245899274	test 0.7221598563	bestTest 0.7221598563	passed: 0.412 sec	total: 24.2s	remaining: 8m 40s
80:	learn 0.7255038722	test 0.7229456668	bestTest 0.7229456668	passed: 0.376 sec	total: 24.6s	remaining: 8m 41s
81:	learn 0.7249747463	test 0.7228334082	bestTest 0.7229456668	passed: 0.248 sec	total: 24.8s	remaining: 8m 40s
82:	learn 0.725070951	test 0.7231701841	bestTest 0.7231701841	passed: 0.268 sec	total: 25.1s	remaining: 8m 39s


148:	learn 0.7426042619	test 0.7436012573	bestTest 0.7437135159	passed: 0.297 sec	total: 43.7s	remaining: 8m 4s
149:	learn 0.7425321083	test 0.7436012573	bestTest 0.7437135159	passed: 0.304 sec	total: 44s	remaining: 8m 4s
150:	learn 0.7427726201	test 0.7436012573	bestTest 0.7437135159	passed: 0.297 sec	total: 44.3s	remaining: 8m 3s
151:	learn 0.742748569	test 0.7436012573	bestTest 0.7437135159	passed: 0.255 sec	total: 44.6s	remaining: 8m 3s
152:	learn 0.7427726201	test 0.7438257746	bestTest 0.7438257746	passed: 0.267 sec	total: 44.8s	remaining: 8m 2s
153:	learn 0.7428447737	test 0.7437135159	bestTest 0.7438257746	passed: 0.279 sec	total: 45.1s	remaining: 8m 2s
154:	learn 0.7429169272	test 0.7441625505	bestTest 0.7441625505	passed: 0.281 sec	total: 45.4s	remaining: 8m 1s
155:	learn 0.7430131319	test 0.7441625505	bestTest 0.7441625505	passed: 0.292 sec	total: 45.7s	remaining: 8m 1s
156:	learn 0.7431093367	test 0.7440502919	bestTest 0.7441625505	passed: 0.272 sec	total: 46s	remaining: 8m


222:	learn 0.749001876	test 0.7488774136	bestTest 0.7488774136	passed: 0.293 sec	total: 1m 5s	remaining: 7m 44s
223:	learn 0.7493145413	test 0.7491019308	bestTest 0.7491019308	passed: 0.312 sec	total: 1m 6s	remaining: 7m 44s
224:	learn 0.7493866949	test 0.7488774136	bestTest 0.7491019308	passed: 0.307 sec	total: 1m 6s	remaining: 7m 44s
225:	learn 0.7493385925	test 0.7487651549	bestTest 0.7491019308	passed: 0.315 sec	total: 1m 6s	remaining: 7m 44s
226:	learn 0.7494107461	test 0.7487651549	bestTest 0.7491019308	passed: 0.237 sec	total: 1m 6s	remaining: 7m 43s
227:	learn 0.7494588484	test 0.7491019308	bestTest 0.7491019308	passed: 0.303 sec	total: 1m 7s	remaining: 7m 43s
228:	learn 0.749531002	test 0.7492141895	bestTest 0.7492141895	passed: 0.3 sec	total: 1m 7s	remaining: 7m 42s
229:	learn 0.7498917697	test 0.7494387068	bestTest 0.7494387068	passed: 0.334 sec	total: 1m 7s	remaining: 7m 42s
230:	learn 0.7497234114	test 0.7494387068	bestTest 0.7494387068	passed: 0.33 sec	total: 1m 8s	remain

295:	learn 0.7550868248	test 0.7533677593	bestTest 0.7533677593	passed: 0.308 sec	total: 1m 29s	remaining: 7m 32s
296:	learn 0.7549425177	test 0.7533677593	bestTest 0.7533677593	passed: 0.295 sec	total: 1m 29s	remaining: 7m 32s
297:	learn 0.7550146712	test 0.753480018	bestTest 0.753480018	passed: 0.309 sec	total: 1m 29s	remaining: 7m 31s
298:	learn 0.755519746	test 0.7535922766	bestTest 0.7535922766	passed: 0.276 sec	total: 1m 29s	remaining: 7m 31s
299:	learn 0.7554716437	test 0.7535922766	bestTest 0.7535922766	passed: 0.324 sec	total: 1m 30s	remaining: 7m 31s
300:	learn 0.7554475925	test 0.7535922766	bestTest 0.7535922766	passed: 0.313 sec	total: 1m 30s	remaining: 7m 30s
301:	learn 0.7555437972	test 0.7537045352	bestTest 0.7537045352	passed: 0.293 sec	total: 1m 30s	remaining: 7m 30s
302:	learn 0.7554716437	test 0.7538167939	bestTest 0.7538167939	passed: 0.284 sec	total: 1m 31s	remaining: 7m 30s
303:	learn 0.7559045649	test 0.7538167939	bestTest 0.7538167939	passed: 0.255 sec	total: 1m

...

952:	learn 0.7829140411	test 0.7800853166	bestTest 0.7803098339	passed: 0.31 sec	total: 4m 52s	remaining: 4m 20s
953:	learn 0.7829861946	test 0.7798607993	bestTest 0.7803098339	passed: 0.267 sec	total: 4m 53s	remaining: 4m 20s
954:	learn 0.7829621434	test 0.7798607993	bestTest 0.7803098339	passed: 0.266 sec	total: 4m 53s	remaining: 4m 19s
955:	learn 0.7828899899	test 0.7798607993	bestTest 0.7803098339	passed: 0.275 sec	total: 4m 53s	remaining: 4m 19s
956:	learn 0.7829140411	test 0.7797485406	bestTest 0.7803098339	passed: 0.277 sec	total: 4m 54s	remaining: 4m 18s
957:	learn 0.783034297	test 0.779636282	bestTest 0.7803098339	passed: 0.249 sec	total: 4m 54s	remaining: 4m 18s
958:	learn 0.7830583482	test 0.7797485406	bestTest 0.7803098339	passed: 0.281 sec	total: 4m 54s	remaining: 4m 18s
959:	learn 0.7829861946	test 0.7798607993	bestTest 0.7803098339	passed: 0.296 sec	total: 4m 54s	remaining: 4m 17s
960:	learn 0.7831064505	test 0.7799730579	bestTest 0.7803098339	passed: 0.284 sec	total: 4m

1025:	learn 0.7849102891	test 0.7816569376	bestTest 0.7818814549	passed: 0.58 sec	total: 5m 17s	remaining: 3m 59s
1026:	learn 0.7849824426	test 0.7816569376	bestTest 0.7818814549	passed: 0.523 sec	total: 5m 17s	remaining: 3m 59s
1027:	learn 0.7848862379	test 0.7816569376	bestTest 0.7818814549	passed: 0.524 sec	total: 5m 18s	remaining: 3m 58s
1028:	learn 0.7848621867	test 0.7816569376	bestTest 0.7818814549	passed: 0.526 sec	total: 5m 18s	remaining: 3m 58s
1029:	learn 0.784765982	test 0.7816569376	bestTest 0.7818814549	passed: 0.498 sec	total: 5m 19s	remaining: 3m 58s
1030:	learn 0.7848381356	test 0.7816569376	bestTest 0.7818814549	passed: 0.481 sec	total: 5m 19s	remaining: 3m 58s
1031:	learn 0.7848140844	test 0.7816569376	bestTest 0.7818814549	passed: 0.433 sec	total: 5m 20s	remaining: 3m 58s
1032:	learn 0.7848862379	test 0.7818814549	bestTest 0.7818814549	passed: 0.504 sec	total: 5m 20s	remaining: 3m 58s
1033:	learn 0.7848862379	test 0.7817691962	bestTest 0.7818814549	passed: 0.536 sec

1097:	learn 0.7870508442	test 0.7835653345	bestTest 0.7837898518	passed: 0.615 sec	total: 5m 57s	remaining: 3m 48s
1098:	learn 0.787026793	test 0.7837898518	bestTest 0.7837898518	passed: 0.463 sec	total: 5m 57s	remaining: 3m 48s
1099:	learn 0.787026793	test 0.7839021105	bestTest 0.7839021105	passed: 0.493 sec	total: 5m 58s	remaining: 3m 47s
1100:	learn 0.7869065371	test 0.7836775932	bestTest 0.7839021105	passed: 0.541 sec	total: 5m 58s	remaining: 3m 47s
1101:	learn 0.7869546395	test 0.7835653345	bestTest 0.7839021105	passed: 0.569 sec	total: 5m 59s	remaining: 3m 47s
1102:	learn 0.7870027418	test 0.7839021105	bestTest 0.7839021105	passed: 0.537 sec	total: 5m 59s	remaining: 3m 47s
1103:	learn 0.7870748954	test 0.7839021105	bestTest 0.7839021105	passed: 0.638 sec	total: 6m	remaining: 3m 47s
1104:	learn 0.7871470489	test 0.7839021105	bestTest 0.7839021105	passed: 0.485 sec	total: 6m	remaining: 3m 46s
1105:	learn 0.787291356	test 0.7840143691	bestTest 0.7840143691	passed: 0.484 sec	total: 6

1169:	learn 0.7890951946	test 0.7849124383	bestTest 0.7849124383	passed: 0.417 sec	total: 6m 32s	remaining: 3m 31s
1170:	learn 0.7890711434	test 0.7849124383	bestTest 0.7849124383	passed: 0.442 sec	total: 6m 33s	remaining: 3m 31s
1171:	learn 0.7890951946	test 0.7848001796	bestTest 0.7849124383	passed: 0.492 sec	total: 6m 33s	remaining: 3m 30s
1172:	learn 0.7890951946	test 0.7849124383	bestTest 0.7849124383	passed: 0.501 sec	total: 6m 34s	remaining: 3m 30s
1173:	learn 0.7891913993	test 0.7849124383	bestTest 0.7849124383	passed: 0.534 sec	total: 6m 34s	remaining: 3m 30s
1174:	learn 0.7892154505	test 0.7850246969	bestTest 0.7850246969	passed: 0.545 sec	total: 6m 35s	remaining: 3m 30s
1175:	learn 0.7891913993	test 0.7850246969	bestTest 0.7850246969	passed: 0.484 sec	total: 6m 35s	remaining: 3m 29s
1176:	learn 0.7892154505	test 0.7850246969	bestTest 0.7850246969	passed: 0.581 sec	total: 6m 36s	remaining: 3m 29s
1177:	learn 0.7891913993	test 0.7852492142	bestTest 0.7852492142	passed: 0.514 s

1241:	learn 0.7910914426	test 0.7867085766	bestTest 0.7870453525	passed: 0.587 sec	total: 7m 9s	remaining: 3m 13s
1242:	learn 0.7910433402	test 0.786259542	bestTest 0.7870453525	passed: 0.541 sec	total: 7m 10s	remaining: 3m 12s
1243:	learn 0.7910433402	test 0.7867085766	bestTest 0.7870453525	passed: 0.473 sec	total: 7m 10s	remaining: 3m 12s
1244:	learn 0.7908990331	test 0.7867085766	bestTest 0.7870453525	passed: 0.538 sec	total: 7m 11s	remaining: 3m 12s
1245:	learn 0.7909471355	test 0.7864840593	bestTest 0.7870453525	passed: 0.522 sec	total: 7m 11s	remaining: 3m 12s
1246:	learn 0.791019289	test 0.7864840593	bestTest 0.7870453525	passed: 0.543 sec	total: 7m 12s	remaining: 3m 11s
1247:	learn 0.791139545	test 0.7867085766	bestTest 0.7870453525	passed: 0.47 sec	total: 7m 12s	remaining: 3m 11s
1248:	learn 0.7911876473	test 0.7867085766	bestTest 0.7870453525	passed: 0.466 sec	total: 7m 13s	remaining: 3m 11s
1249:	learn 0.7911876473	test 0.7868208352	bestTest 0.7870453525	passed: 0.55 sec	tot

In [ ]:
clf.score(xcv, ycv)

In [ ]:
clf.score(xva, yva)